In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import pandas as pd

In [2]:
#! lấy data đã được xử lý (lúc này phải chạy ipynb trong datapreprocessing) cho vào all_movie_link:
all_movie_link = []
with open('../DataPreprocessing/processed_movie_link.txt','r',encoding="utf8") as file:
    all_movie_link = file.readlines()
all_movie_link.pop(0)

'https://myanimelist.net/anime/season?_location=mal_h_m\n'

In [3]:

all_movie_link[1]

'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood\n'

In [4]:
#! crawl user name ok. Chỉ cần thêm link /reviews ở cuối rồi crawl là oke. làm sạch thì để sau!
cnt = 0
user_name = []
my_user_name =[]
post_name = []
id = 0


film_cmt = []
my_user_name = []

my_film_name = []
#url = "https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood\n/reviews"
for url in all_movie_link:
    url = url.replace('\n','')
    url = url +  "/reviews"
    my_film_name.append(url)
    


    page = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(page.text,'html')
    user_name = soup.find_all('a',href=True)
    sleep(2)
    my_user_name = []
    for name in user_name:
        if 'https://myanimelist.net/profile/' in name['href']:
            temp = name['href']
            my_user_name.append(temp)
            #print(temp)
    film_cmt.append(my_user_name)
    cnt += 1
    if cnt == 200:
        break




In [5]:
df_user_name = pd.DataFrame({'my_film_cmt': film_cmt})
df_user_name.to_csv('./film_cmt_test1.csv')

In [6]:
#! dòng cần thêm: 
print(film_cmt[-1])

['https://myanimelist.net/profile/Hachikawa', 'https://myanimelist.net/profile/Hachikawa', 'https://myanimelist.net/profile/MizuhoKamui', 'https://myanimelist.net/profile/MizuhoKamui']


In [7]:
#! remove uneseccasy and duplicate
new = []
new_cmt = []
for my_user_name in film_cmt:
    for name in my_user_name:
        name = name.replace('https://myanimelist.net/profile/','')
        new.append(name)
    new_cmt.append(new)
    new = []
#! remove duplicate
#print(new_cmt)
my_user = []
my_user_name = []

for new in new_cmt:
    for name in new:
        if name not in my_user_name:
            my_user_name.append(name)
    my_user.append(my_user_name)
    my_user_name = []
print(len(my_user))

200


In [8]:
# my_user[0]

In [9]:
#! in ra tên của bộ phim đó bằng với số lượng user_cmt: 
multiple_post_name = []
post_name = []
cnt = 0
for movie_name in my_film_name:
    movie_name = movie_name.replace('https://myanimelist.net/anime/','')
    key = movie_name.index('/')
    movie_name = movie_name[key+1:-8]
    #print("my movie name at "+str(cnt) + movie_name)
    for i in range(len(my_user[cnt])):
        post_name.append(movie_name)
    
    cnt += 1
    multiple_post_name.append(post_name)
    post_name = []
    if cnt == 200:
        break

In [10]:
# multiple_post_name[0]

In [11]:
# print(len(multiple_post_name[1]))
# print(len(my_user[1]))

In [12]:
#! crawl cmt của người review
my_cmt = []
my_cmt_with_film = []
cnt = 0
for url in my_film_name:
    
    page = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(page.text,'html')
    user_cmt = soup.find_all(class_='text')
    sleep(2)
    for cmt  in user_cmt:
        a = cmt.get_text().replace('\n','')
            #print(a)
        my_cmt.append(a)
    my_cmt_with_film.append(my_cmt)
    my_cmt = []
    cnt += 1
    if cnt == 200:
        break

In [13]:
# len(my_cmt_with_film)

In [14]:
#! ok!
#! crawl recommend link của người review
my_recommend = []
#url = "https://myanimelist.net/anime/52991/Sousou_no_Frieren/reviews"
my_recommend_film = []
cnt = 0
for url in my_film_name:
    page = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(page.text,'html')
    user_cmt = soup.find_all(class_='tags')
    sleep(2)
    for cmt in user_cmt:
        a = cmt.get_text().replace('\n','')
            #print(a)
        my_recommend.append(a)
    my_recommend_film.append(my_recommend)
    my_recommend = []
    cnt += 1
    if cnt == 200:
        break

In [15]:

my_episode = []
my_episode_film = []
for my_recommend in my_recommend_film:
    for cmt in my_recommend:
        if "(" in cmt:
            key = cmt.index("(")
            my_episode.append(cmt[key+1:key+9])
        else:
            my_episode.append("done")
    my_episode_film.append(my_episode)
    my_episode = []
len(my_episode_film)


200

In [16]:
#! crawl number of reaction with class num js-num    data-pagename  anime-title-reviews  data-id



In [17]:
from itertools import chain
multiple_post_name = list(chain.from_iterable(multiple_post_name))
my_user = list(chain.from_iterable(my_user))
my_cmt_with_film = list(chain.from_iterable(my_cmt_with_film))
my_recommend_film = list(chain.from_iterable(my_recommend_film))
my_episode_film = list(chain.from_iterable(my_episode_film))

In [18]:
len(multiple_post_name)

3660

In [19]:
len(my_user)

3660

In [20]:
len(my_cmt_with_film)

3660

In [21]:
len(my_recommend_film)

3660

In [22]:
len(my_episode_film)

3660

In [23]:
#! append hết toàn bộ data vào csv dạng: username - comment
#! cho thêm một trường là post name

df_cmt = pd.DataFrame({'Post name': multiple_post_name,'Username': my_user,'Comment':my_cmt_with_film,'User feeling': my_recommend_film,'user_see_episode': my_episode_film})
df_cmt

,Post name,Username,Comment,User feeling,user_see_episode
0,Sousou_no_Frieren,Czekaj,"With lives so short, why do we...",RecommendedPreliminary ...,5/28 eps
1,Sousou_no_Frieren,Shyra_,Frieren is the most overrated ...,Not RecommendedFunnyPreliminary ...,3/28 eps
2,Sousou_no_Frieren,chekkit,I feel so catered to.\rIt feel...,Recommended,done
3,Sousou_no_Frieren,Trikkiez,Style-\rFrieren doesn't have i...,Not RecommendedFunny,done
4,Sousou_no_Frieren,TheRealist68,"Through 3 episodes, Frieren ap...",Mixed FeelingsPreliminary ...,6/28 eps
...,...,...,...,...,...
3655,Kimetsu_no_Yaiba__Mugen_Ressha-hen,CyberXane,I hate rating this a 10 becaus...,Recommended,done
3656,Kimetsu_no_Yaiba__Mugen_Ressha-hen,meg_lowde,This season is literally just ...,Mixed Feelings,done
3657,Kimetsu_no_Yaiba__Mugen_Ressha-hen,Typo_comma,I am more concerned about the ...,Recommended,done
3658,Made_in_Abyss_Movie_2__Hourou_Suru_Tasogare,Hachikawa,Made in Abyss is a spectacular...,Recommended,done


In [24]:
df_cmt.to_csv('./my_post_name_and_cmt')

In [2]:
df_cmt = pd.read_csv('./my_post_name_and_cmt.csv')
df_cmt.head(3)

,Unnamed: 0,Post name,Username,Comment,User feeling,user_see_episode
0,0,Sousou_no_Frieren,Czekaj,"With lives so short, why do we...",RecommendedPreliminary ...,5/28 eps
1,1,Sousou_no_Frieren,Shyra_,Frieren is the most overrated ...,Not RecommendedFunnyPreliminary ...,3/28 eps
2,2,Sousou_no_Frieren,chekkit,I feel so catered to.\rIt feel...,Recommended,done
